In [1]:
import tensorflow as tf
from tensorflow import keras

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.metrics import mean_squared_error

import numpy as np
import DataPreprocessing
import SmartPolynomials

Using TensorFlow backend.


In [2]:
combs = ['1,3','1,4','2,3','2,4','1,2,3','1,2,4','1,3,4','2,3,4','1,2,3','1,2,4','1,3,4','2,3,4']

In [9]:
me_train, mom_train = DataPreprocessing.npy('LO/MG_uuuxux/me_4jet_1000000', 'LO/MG_uuuxux//mom_4jet_1000000', combs, frac=1)
me_test, mom_test = DataPreprocessing.npy('LO/MG_uuuxux/me_4jet_100000', 'LO/MG_uuuxux/mom_4jet_100000', combs, frac=0.1)
me_train=me_train/10**50
me_test=me_test/10**50

In [10]:
# Random Model from Tensorflow Website
def build_model(components=3000):
  model = keras.Sequential([
    keras.layers.Dense(components, activation='relu', input_shape=[9,]), #could try swish??  
    
    keras.layers.Dense(components, activation='relu'),
    keras.layers.Dense(components, activation='relu'),
    keras.layers.Dense(components, activation='relu'),   
    keras.layers.Dense(components, activation='relu'),
      
    keras.layers.Dense(1) 
  ])

  sgd=tf.keras.optimizers.SGD(0.00000000000005, momentum=0.95, nesterov=True)
  model.compile(loss='mse', 
                optimizer=sgd,
                metrics=['mse'])
  return model

In [ ]:
step=5000

#model = build_model()

scaler = StandardScaler()

training_error = []
test_error = []
component = []

for n in range(0,len(mom_train),step):
    poly_train = SmartPolynomials.smarterPolynomialFeatures(4, mom_train[n:n+step])
    ##Rescale
    scaler.partial_fit(poly_train) #Rescale on training set
    poly_train = scaler.transform(poly_train)
    
    model.fit(poly_train,me_train[n:n+step], batch_size=1, epochs=1, shuffle=True)
        
    if(n%20000==0): 
        print(n)
    
        ##Testing Error
        lin_mse = 0
        lin_perc = 0
        me_predictions = np.zeros(len(me_test))
        for m in range(0,len(mom_test),200):
            poly_test = SmartPolynomials.smarterPolynomialFeatures(4, mom_test[m: m+step])
            poly_test = scaler.transform(poly_test)

            me_predict = model.predict(poly_test, verbose=0) #Prediction on test set
            me_predictions[m:m+step] = me_predict.T
            lin_mse += np.sum(mean_squared_error(me_test[m:m+step], me_predict))
            lin_perc += np.sum(100*np.divide(np.abs(me_test[m:m+step] - me_predict), me_test[m:m+step]))
        test_error.append(np.sqrt(lin_mse*step/len(me_test)))
        
        u=model.evaluate(poly_train, me_train[n:n+step])[0]
        training_error.append(np.sqrt(u))
              
        print(lin_perc/len(me_test))
        print(training_error)
        print(test_error)

Epoch 1/1
5000/5000 [==============================]5000/5000 [==============================] - 509s 102ms/step - loss: 643189373.1886 - mean_squared_error: 643189373.1886 - los

0
5000/5000 [==============================]5000/5000 [==============================] - 4s 751us/step

8460508.993762197
[25361.178335779274]
[126038.36221901342]
Epoch 1/1
5000/5000 [==============================]5000/5000 [==============================] - 495s 99ms/step - loss: 639423830.9498 - mean_squared_error: 639423830.9498

Epoch 1/1
5000/5000 [==============================]5000/5000 [==============================] - 495s 99ms/step - loss: 643914133.0333 - mean_squared_error: 643914133.0333 [===========

Epoch 1/1
5000/5000 [==============================]5000/5000 [==============================] - 495s 99ms/step - loss: 628010229.9801 - mean_squared_error: 628010229.9801

Epoch 1/1
5000/5000 [==============================]5000/5000 [==============================] - 495s 99ms/step - loss: 6454

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



5000/5000 [==============================]5000/5000 [==============================] - 493s 99ms/step - loss: 634596062.3906 - mean_squared_error: 634596062.3906

60000
5000/5000 [==============================]5000/5000 [==============================] - 4s 744us/step

8460435.950259233
[25361.178335779274, 25405.97957709956, 25155.964575074435, 25191.189943914917]
[126038.36221901342, 126038.3560806635, 126038.34956439528, 126038.34320265806]
Epoch 1/1
 498/5000 [=>............................] 498/5000 [=>............................] - ETA: 7:22 - loss: 733465498.7769 - mean_squared_error: 733465498.7769

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



5000/5000 [==============================]5000/5000 [==============================] - 496s 99ms/step - loss: 669916342.4479 - mean_squared_error: 669916342.4479

Epoch 1/1
5000/5000 [==============================]5000/5000 [==============================] - 498s 100ms/step - loss: 628765783.6075 - mean_squared_error: 628765783.6075

Epoch 1/1
5000/5000 [==============================]5000/5000 [==============================] - 496s 99ms/step - loss: 653914803.1302 - mean_squared_error: 653914803.1302

Epoch 1/1
5000/5000 [==============================]5000/5000 [==============================] - 495s 99ms/step - loss: 641603622.3556 - mean_squared_error: 641603622.3556

80000
5000/5000 [==============================]5000/5000 [==============================] - 4s 725us/step

8460411.345622517
[25361.178335779274, 25405.97957709956, 25155.964575074435, 25191.189943914917, 25329.895584403817]
[126038.36221901342, 126038.3560806635, 126038.34956439528, 126038.34320265806, 126038.3367

...

3588/5000 [====================>.........]3588/5000 [====================>.........] - ETA: 2:24 - loss: 644342747.0763 - mean_squared_error: 644342747.076381/5000 [====================>.........] - ETA: 2:25 - loss: 644163491.2138 - mean_squared_error: 

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



5000/5000 [==============================]5000/5000 [==============================] - 508s 102ms/step - loss: 638686740.3154 - mean_squared_error: 638686740.3154 - loss: 638877793.6376 - mean_squared_error: 638877793.

Epoch 1/1
1562/5000 [========>.....................]1562/5000 [========>.....................] - ETA: 5:51 - loss: 647041592.2130 - mean_squared_error: 647041592.2130

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



5000/5000 [==============================]5000/5000 [==============================] - 548s 110ms/step - loss: 625761236.2753 - mean_squared_error: 625761236.2753

Epoch 1/1
5000/5000 [==============================]5000/5000 [==============================] - 568s 114ms/step - loss: 631528032.6973 - mean_squared_error: 631528032.6973

760000
5000/5000 [==============================]5000/5000 [==============================] - 4s 710us/step

8459582.108598808
[25361.178335779274, 25405.97957709956, 25155.964575074435, 25191.189943914917, 25329.895584403817, 25225.369124403314, 25308.716433292306, 24437.233664455558, 25307.261893835926, 25389.080943240147, 24999.960921057456, 25175.409233106817, 25323.75875342363, 25141.750142549743, 24811.5573668724, 25438.98972383927, 24904.378470879372, 24627.78894306186, 25179.0980459269, 25358.192187930115, 25436.326352113036, 24865.99078053396, 25358.798124201392, 25156.3346208942, 25837.194109453914, 24862.997164847202, 25020.791810716142, 25188